## Scrapping Data Using Product URL

In [1]:
import requests
from bs4 import BeautifulSoup

<b>1. Flipkart

In [2]:
# URL of the product
url = 'https://www.flipkart.com/godrej-7-kg-5-star-rain-shower-spin-washing-machine-semi-automatic-top-load-grey-white/p/itm8d4974369ec36?pid=WMNGHNFWBHQWGKDS&lid=LSTWMNGHNFWBHQWGKDSUZZTOB&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_1&otracker=nmenu_sub_TVs%20%26%20Appliances_0_Semi%20Automatic%20Top%20Load&fm=search-autosuggest&iid=en_hVlTg4p2fWuHBCZztsP5C4rklUR8x7y8bY6SU2Nil7efGyNjNd9U9Egz6ydAd_VwVHFQ07xCI76A6oQ8GLNDMfUFjCTyOHoHZs-Z5_PS_w0%3D&ppt=sp&ppn=sp&ssid=tx2trlohmo0000001722806482398'

# Send a GET request
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract title
title_element = soup.find('span', {'class': 'VU-ZEz'})
title = title_element.get_text() if title_element else 'Not Available'

# Extract brand
brand_element = soup.find('span', {'class': 'mEh187'})
brand = brand_element.get_text() if brand_element else 'Not Applicable'

# Extract price
price_element = soup.find('div', {'class': 'Nx9bqj CxhGGd'})
price = price_element.get_text() if price_element else 'Not Available'

# Check stock availability
stock_button = soup.find('button', {'class': 'QqFHMw vslbG+ _3Yl67G _7Pd1Fp'})
stock_availability = 'In Stock' if stock_button else 'Out of Stock'

# Print the results
print(f"Title: {title}")
print(f"Brand: {brand}")
print(f"Price: {price}")
print(f"Stock Availability: {stock_availability}")

Title: Godrej 7 kg 5 Star With Rain Shower Spin Washing Machine Semi Automatic Top Load Grey, White  (WS EDGE CLS 70 5.0 PN2 GPGR)
Brand: Not Applicable
Price: ₹9,990
Stock Availability: In Stock


<b>2. Amazon Product Scraper

In [4]:
# URL of the product
url = 'https://www.amazon.in/Pampers-Premium-Care-Diapers-Medium/dp/B07F2X8BHM/ref=sr_1_1_sspa?crid=L3M7627TI1NW&dib=eyJ2IjoiMSJ9.DrnWVjdZ1I_Y7tKI6xifyV7PZ18lxbjGALgibRT9xDsn4zkdpVz-bTwCHdEYx3RUpHcKcy9Wr8hLZFhzq-_yt3-eELFrEdOEfI2Dw_mHpoPgH87tVymiditUZiXQhvPRlJ8VjPikpzdpX5SdfJYc_f-UsBrET27l626GY4f7fF-gTvlQQTog1ASR2L3pHEgCvYo8CZGmuyUEBWekacn0k09ZCVBoLYKUcwCxlZd8Pu-ASH2_jnBhYCYbgYzlrMi3sh0lBaEyc-1NBuo-NqTbXVboMsj1geNy71el6t52lV4.PjVip_kpZxwGbl4SrHa1BhJmkK0SQ_LTVDXYnpkamXs&dib_tag=se&keywords=pampers&qid=1722796731&sprefix=pampers%2Caps%2C248&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1'

# Send a GET request
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Extract title
title = soup.find('span', {'id': 'productTitle'}).get_text().strip()

# Extract brand
brand_element = soup.find('a', {'id': 'bylineInfo'})

if brand_element:
    brand_text = brand_element.get_text().strip()
    if "Visit the" in brand_text and "Store" in brand_text:
        brand = brand_text.replace("Visit the ", "").replace(" Store", "").strip()
    elif "Brand:" in brand_text:
        brand = brand_text.replace("Brand: ", "").strip()
    else:
        brand = brand_text

# Extract price
price = soup.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).get_text()

# Check stock availability
stock_info = soup.select_one("span[id='submit.buy-now-announce']").get_text().strip()


print(f"Title: {title}")
print(f"Brand: {brand}")
print(f"Price: {price}")
print(f"Stock Availability: {True if stock_info else False}")

Title: Pampers Premium Care Pant Style Baby Diapers, Medium (M), 54 Count, All-in-1 Diapers with 360 Cottony Softness, 7-12 kg Diapers
Brand: Pampers
Price: ₹914.00
Stock Availability: True


<b>Multiple Page Scrapping

In [6]:
import time
import random
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

headers_list = [
    {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"},
    {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
    {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
    # Add more user-agents if needed
]

def fetch_url(url):
    retries = 5
    delay = 1  # initial delay in seconds
    for i in range(retries):
        try:
            headers = random.choice(headers_list)
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.content
            else:
                print(f"Error fetching URL {url}: {response.status_code} {response.reason}")
        except requests.RequestException as e:
            print(f"Error fetching URL {url}: {e}")
        print(f"Retry {i+1}/{retries}...")
        time.sleep(delay)
        delay *= 2  # exponential backoff
    return None

def parse_content(content):
    soup = BeautifulSoup(content, 'html.parser')
    items = []
    titles = soup.find_all("span", class_="a-size-medium a-color-base a-text-normal")
    prices = soup.find_all("span", class_="a-price-whole")
    for title, price in zip(titles, prices):
        item = {
            "title": title.get_text(strip=True),
            "price": price.get_text(strip=True)
        }
        items.append(item)
    return items

def main():
    base_url = "https://www.amazon.in/s?k=mobile+above+10000&page={page}&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_{page}"
    all_items = []
    for page in tqdm(range(1, 21), desc="Scraping pages"):  # adjust the range as needed
        url = base_url.format(page=page)
        content = fetch_url(url)
        if content:
            items = parse_content(content)
            all_items.extend(items)
        time.sleep(random.uniform(1, 3))  # random delay between requests
    
    # Save to JSON file
    with open('scraped_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_items, f, ensure_ascii=False, indent=4)
    print("Data saved to scraped_data.json")

if __name__ == "__main__":
    main()

Scraping pages:  15%|█▌        | 3/20 [00:11<01:08,  4.05s/it]

Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=4&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_4: 503 Service Unavailable
Retry 1/5...
Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=4&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_4: 503 Service Unavailable
Retry 2/5...
Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=4&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_4: 503 Service Unavailable
Retry 3/5...


Scraping pages:  20%|██        | 4/20 [00:25<02:06,  7.88s/it]

Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=5&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_5: 503 Service Unavailable
Retry 1/5...


Scraping pages:  45%|████▌     | 9/20 [00:47<00:51,  4.70s/it]

...

Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=17&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_17: 503 Service Unavailable
Retry 1/5...


Scraping pages:  90%|█████████ | 18/20 [02:05<00:10,  5.42s/it]

Error fetching URL https://www.amazon.in/s?k=mobile+above+10000&page=19&crid=3F6LBWWE3ABBY&qid=1722799603&sprefix=mobile+abo%2Caps%2C249&ref=sr_pg_19: 503 Service Unavailable
Retry 1/5...


Scraping pages: 100%|██████████| 20/20 [02:14<00:00,  6.74s/it]

Data saved to scraped_data.json


In [8]:
# URL of the product
url = 'https://www.walmart.com/ip/VILINICE-Noise-Cancelling-Headphones-Wireless-Bluetooth-Over-Ear-Headphones-with-Microphone-Black-Q8/249420079'

# Send a GET request
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

<html lang="en">
<head>
<title>Robot or human?</title>
<meta content="width=device-width" name="viewport"/>
<style>
    #sign-in-widget a,
    #sign-in-widget a:active,
    #sign-in-widget a:hover {
        color: #000
    }

    #sign-in-widget h1 {
        font-weight: 500;
        font-size: 20px;
        font-size: 1.25rem;
        letter-spacing: -.6px;
        margin: 1px auto
    }

    @media (min-width:30em) {
        #sign-in-widget h1 {
            margin-top: 24px;
            font-size: 24px;
            font-size: 1.5rem
        }
    }

    #sign-in-widget {
        font-family: BogleWeb, Helvetica Neue, Helvetica, Arial, sans-serif
    }

    #sign-in-widget * {
        box-sizing: border-box
    }

    #sign-in-widget .text-right {
        text-align: right
    }

    @font-face {
        font-family: NewYorkIcons;
        src: url(6255ed72d86ece856725a2d80878bce6.eot);
        font-weight: 400;
        font-style: normal
    }

    @font-face {
        font-family: Ne

In [10]:
import requests
from bs4 import BeautifulSoup

# URL of the product
url = 'https://www.walmart.com/ip/VILINICE-Noise-Cancelling-Headphones-Wireless-Bluetooth-Over-Ear-Headphones-with-Microphone-Black-Q8/249420079'

# Headers to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Send a GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Print the parsed HTML (for debugging purposes)
print(soup.prettify())


<html lang="en">
 <head>
  <title>
   Robot or human?
  </title>
  <meta content="width=device-width" name="viewport"/>
  <style>
   #sign-in-widget a,
    #sign-in-widget a:active,
    #sign-in-widget a:hover {
        color: #000
    }

    #sign-in-widget h1 {
        font-weight: 500;
        font-size: 20px;
        font-size: 1.25rem;
        letter-spacing: -.6px;
        margin: 1px auto
    }

    @media (min-width:30em) {
        #sign-in-widget h1 {
            margin-top: 24px;
            font-size: 24px;
            font-size: 1.5rem
        }
    }

    #sign-in-widget {
        font-family: BogleWeb, Helvetica Neue, Helvetica, Arial, sans-serif
    }

    #sign-in-widget * {
        box-sizing: border-box
    }

    #sign-in-widget .text-right {
        text-align: right
    }

    @font-face {
        font-family: NewYorkIcons;
        src: url(6255ed72d86ece856725a2d80878bce6.eot);
        font-weight: 400;
        font-style: normal
    }

    @font-face {
        fo